In [1]:
import pandas as pd
import numpy as np
import random

from bs4 import BeautifulSoup
import requests
import re

/Users/davidhe/Documents/GitHub/ml-toolbox/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
user_agents = [
    # Chrome on Mac
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/139.0.0.0 Safari/537.36",
    # Chrome on Windows
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/139.0.0.0 Safari/537.36",
    # Firefox on Mac
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:130.0) Gecko/20100101 Firefox/130.0",
    # Firefox on Windows
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:130.0) Gecko/20100101 Firefox/130.0",
    # Edge on Windows
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/139.0.0.0 Safari/537.36 Edg/139.0.0.0",
]

headers = {
    "User-Agent": random.choice(user_agents)
}

url = "https://www.swimrankings.net/index.php?page=rankingDetail&club=CAN&season=-1&course=LCM"

page = requests.get(url, headers=headers)


In [ ]:
# used my brain and hard work to locate that the table is here
soup = BeautifulSoup(page.text[16000:63000], "html.parser")

In [4]:
content = soup.prettify()
temp = BeautifulSoup(content, "html.parser")
rows = temp.find_all("tr")

In [5]:
for row in rows[1:]:
    columns = row.find_all("td")
    data = [col.text.strip().replace('\xa0', ' ') for col in columns]
    print(data)


['Canada', 'Long Course (50m)']
['Men, Open', 'Top Times', 'Alltime']
['Last built 26 Aug 2025 - 2:39']
['Top Times\n           \n\n            Freestyle\n           \n\n            Backstroke\n           \n\n            Breaststroke\n           \n\n            Butterfly\n           \n\n            Medley\n           \n\n            Relays\n           \n\n            Statistic\n           \n\n            Custom\n           \n\n            Meets', 'Men\n           \n\n            Women', 'Long Course (50m)\n           \n\n            Short Course (25m)', 'Alltime\n           \n\n            2025\n           \n\n            2024\n           \n\n            2023\n           \n\n            2022\n           \n\n            2021\n           \n\n            2020\n           \n\n            2019\n           \n\n            2018\n           \n\n            2017\n           \n\n            2016\n           \n\n            2015\n           \n\n            2014\n           \n\n            2013\n 

In [6]:
data = []

# Extract data from rows and append to the data list
for row in rows[1:]:
    columns = row.find_all("td")
    if len(columns) == 9:  # Ensure the row has the correct number of columns
        data.append([col.text.strip().replace('\xa0', ' ') for col in columns])
    
    elif len(columns) == 8:
        # the relays don't have Date of Birth. Just fill it with "N/A"
        # Clean up the names by removing unwanted '\n' and extra spaces
        columns[1].string = re.sub(r'\s+', ' ', columns[1].text.strip())
        row_data = [col.text.strip().replace('\xa0', ' ') for col in columns]
        row_data.insert(2, "N/A")  # Insert "N/A" at
        data.append(row_data)

# Create the DataFrame
df = pd.DataFrame(data, columns=["Event", "Name(s)", "Year of Birth", "Country", "Club", "Time", "Points", "Date", "City (Nation)"])

print(df)

                    Event                                            Name(s)  \
0                                                                              
1           50m Freestyle                                     LIENDO, Joshua   
2          100m Freestyle                                      HAYDEN, Brent   
3          200m Freestyle                                     SAUVE, Antoine   
4          400m Freestyle                                     COCHRANE, Ryan   
5          800m Freestyle                                     COCHRANE, Ryan   
6         1500m Freestyle                                     COCHRANE, Ryan   
7          50m Backstroke                                    ACEVEDO, Javier   
8         100m Backstroke                                     TIERNEY, Blake   
9         200m Backstroke                                     TIERNEY, Blake   
10       50m Breaststroke                                     DICKENS, Scott   
11      100m Breaststroke               

In [7]:
df.to_csv("swim_rankings.csv", index=False)